In [9]:
import sqlite3
import random
import itertools
import subprocess
import os
import shlex
import time
import pickle

In [10]:
import bz2

In [11]:
from PIL import Image, ImageDraw, ImageFont

# from pillow import Image

import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import matplotlib.cm as cm
from matplotlib.colors import Normalize

In [12]:
import keras
from keras.preprocessing import image
from keras.applications.imagenet_utils import decode_predictions, preprocess_input
from keras.models import Model, load_model

In [13]:
# this seems to help with some GPU memory issues

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
session = tf.Session(config=config)

In [6]:
# load ternary classifier
# model = keras.applications.VGG16(weights='imagenet', include_top=True)
model = load_model('checkpoints/ternary_20190911_9748x/diagram-sensor-unsure_vgg16-2000.hdf5')
print("model loaded")
model.summary()

W0729 12:26:00.910594 140487372396352 deprecation_wrapper.py:119] From /home/rte/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:517: The name tf.placeholder is deprecated. Please use tf.compat.v1.placeholder instead.

W0729 12:26:00.918291 140487372396352 deprecation_wrapper.py:119] From /home/rte/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:4138: The name tf.random_uniform is deprecated. Please use tf.random.uniform instead.

W0729 12:26:00.946867 140487372396352 deprecation_wrapper.py:119] From /home/rte/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:3976: The name tf.nn.max_pool is deprecated. Please use tf.nn.max_pool2d instead.

W0729 12:26:01.129150 140487372396352 deprecation_wrapper.py:119] From /home/rte/.local/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:174: The name tf.get_default_session is deprecated. Please use tf.compat.v1.get_default_session instead.

W0729 12:26:01.129934 1404

model loaded
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0       

In [7]:
def load_image(path):
    img = image.load_img(path, target_size=model.input_shape[1:3])
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return img, x

In [14]:
# set up the feature extractor -- only needed if doing something with second last layer

model = keras.applications.VGG16(weights='imagenet', include_top=True)
feat_extractor = Model(inputs=model.input, outputs=model.get_layer("fc2").output)
print("feature extractor setup")
feat_extractor.summary()

feature extractor setup
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)    

### Analyse samples and save csv

In [ ]:
current_path = "/home/rte/data/images/random/100k/classification/"
# current_path = convert_path

image_extensions = ['.jpg', '.png', '.jpeg']   # case-insensitive (upper/lower doesn't matter)
max_num_images = 100000

images = [os.path.join(dp, f) for dp, dn, filenames in os.walk(current_path) for f in filenames if os.path.splitext(f)[1].lower() in image_extensions]
num_x = len(images)
print("keeping %d images to analyze" % num_x)

In [ ]:
start = time.time()

all_predictions = []

features = []

writefile = "predictions_100k.txt"
f = open(writefile, "a+")
        
for i, image_path in enumerate(images):
    if i % 500 == 0:
        end = time.time()
        elap = end - start;
        print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
        start = time.time()
    img, x = load_image(image_path)
    
    f.write(os.path.basename(image_path))

#     feat = feat_extractor.predict(x)[0]
#     features.append(feat)
    
    predictions = model.predict(x)
    
#     print(predictions)
#     print(decode_predictions(predictions))
    
    for _, pred, prob in decode_predictions(predictions)[0]:
#         print("predicted %s with probability %0.3f" % (pred, prob))
        f.write("," + str(pred) + "," + str(prob))
                
    f.write("\n")
        
print('finished predicting class for %d images' % len(images))

f.close()

In [ ]:
# write images, predictions to a pickle file

f = "classification_vgg_subset" + ".pickle"

print(f)

# WRITE
with open(f, "wb") as write_file:
    pickle.dump([images, features], write_file)
    write_file.close()


### Decode predictions from pickle

In [ ]:
p = "/home/rte/re-imaging/visualisation/features/features_0_50000_vgg.pkl.pbz2"
predictions = []
with bz2.open(p, "r") as read_file:
    predictions = pickle.load(read_file)
for p in predictions[:5]:
    print(p)

In [ ]:
p = "/home/rte/re-imaging/visualisation/features/v1/features_0_50000_vgg.pkl.pbz2"
# predictions = []
with bz2.open(p, "r") as read_file:
    predictions = pickle.load(read_file)

writefile = "predictions_1m.txt"
f = open(writefile, "a+")

for pred in predictions[:1]:
    for _, pred, prob in decode_predictions(pred):
        print("predicted %s with probability %0.3f" % (pred, prob))
        f.write("," + str(pred) + "," + str(prob))

        f.write("\n")
        print('finished predicting class for %d images' % len(images))

f.close()

In [ ]:
len(predictions[0])

In [ ]:
image_folder = "/mnt/hd2/images/all/"
for i, image_path in enumerate(filepaths[:1]):
    img, x = load_image(image_folder + image_path)

    predictions = model.predict(x)
    print(predictions)

In [ ]:
len(predictions[0])

In [ ]:
print(predictions)

In [ ]:
predictions

In [ ]:
for _, pred, prob in decode_predictions(predictions)[0]:
        print("predicted %s with probability %0.3f" % (pred, prob))

In [ ]:
print(decode_predictions(predictions))

### Save predictions images

In [15]:
filepaths = []

with open("/home/rte/data/paths/all_converted_jpg_images_shuf.txt", "r") as f:
    lines = f.readlines()
    print("length:",len(lines))
    print(lines[0])
for l in lines:
    # substrings = l.rsplit(",", 1)
    filepaths.append(l.strip())
    # image_ids.append(substrings[1].strip())

length: 10052503
2971229.jpg



In [ ]:
image_folder = "/mnt/hd2/images/all/"

plt.close('all')

for i, image_path in enumerate(filepaths[0:1000]):
    plt.close('all')

    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 5)
    fig.set_dpi(150)

    img, x = load_image(image_folder + image_path)
    im = mpimg.imread(image_folder + image_path)
    pim = Image.open(image_folder + image_path)
    width, height = pim.size
#     print(width, height)
    asp = width/height
    
    prediction = model.predict(x)
    
    classes = []
    probabilities = []
    
    for i, (_, pred, prob) in enumerate(decode_predictions(prediction)[0]):
#         print("predicted %s with probability %0.3f" % (pred, prob))
        classes.append(pred)
        probabilities.append(prob)
        
    imgplot = ax[0].imshow(im, aspect=asp)
    ax[0].axis('off')
    ax[0].set_title("Image ID: " + image_path.split(".")[0])

    y_pos = np.arange(len(classes))

#     my_cmap = cm.get_cmap('jet')
#     my_norm = Normalize(vmin=0, vmax=8)
    
    probabilities = np.array(probabilities)
#     color = np.array(['rgb(255,255,255)']*probabilities.shape[0])
#     colors = cm.winter(probabilities)

    ax[1].set_xlim(0, 1)
    ax[1].barh(y_pos, probabilities, align='center', color='0.75')

#     ax[1].barh(y_pos, probabilities, align='center', color=my_cmap(my_norm(probabilities)))
    ax[1].set_yticks(y_pos)
    ax[1].set_yticklabels(classes)
    ax[1].invert_yaxis()
    ax[1].set_title('VGG16 Prediction')
    ax[1].set_xlabel('Percentage')
    
#     plt.subplots_adjust(wspace=0.5)
    plt.tight_layout(pad=5.0)

#     plt.axis('off')
#     target = "predictions/" + classes[0]
    target = "predictions"
#     print(target)
    if not os.path.exists(target): os.mkdir(target)
        
    save_path = target + "/vgg16_prediction_" + classes[0] + "_" + \
                        "{:d}".format(int(probabilities[0]*100)) + "pc_" + \
                        image_path
    print(save_path)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
# bbox_inches='tight'

### Save images for results form Ternary Classifier


In [ ]:
image_folder = "/mnt/hd2/images/all/"

classes = ["diagram", "sensor", "mixed"]

for i, image_path in enumerate(filepaths[0:1000]):
    plt.close('all')

    fig, ax = plt.subplots(1, 2)
    fig.set_size_inches(10, 5)
    fig.set_dpi(150)

    img, x = load_image(image_folder + image_path)
    im = mpimg.imread(image_folder + image_path)
    pim = Image.open(image_folder + image_path)
    width, height = pim.size
#     print(width, height)
    asp = width/height
    
    prediction = model.predict(x)
    
    probabilities = []
    
    for i, prob in enumerate(prediction[0]):
#         print("predicted %s with probability %0.3f" % (pred, prob))
#         classes.append(pred)
        probabilities.append(prob)
        
    imgplot = ax[0].imshow(im, aspect='auto')
    ax[0].axis('off')
    ax[0].set_title("Image ID: " + image_path.split(".")[0])

    y_pos = np.arange(len(classes))

    my_cmap = cm.get_cmap('jet')
    my_norm = Normalize(vmin=0, vmax=8)
    
    probabilities = np.array(probabilities)
#     color = np.array(['rgb(255,255,255)']*probabilities.shape[0])
#     colors = cm.winter(probabilities)

    ax[1].set_xlim(0, 1)
    ax[1].barh(y_pos, probabilities, align='center', color='0.75')
    for i, v in enumerate(probabilities):
        ax[1].text(-0.12,i+0.25, "{:.2f}".format(v), color='0.5', va='center')
#         color='blue', fontweight='bold'

#     ax[1].barh(y_pos, probabilities, align='center', color=my_cmap(my_norm(probabilities)))
    ax[1].set_yticks(y_pos)
    ax[1].set_yticklabels(classes)
    ax[1].invert_yaxis()
    ax[1].set_title('Ternary Prediction')
    ax[1].set_xlabel('Percentage')
    
#     plt.subplots_adjust(wspace=0.5)
    plt.tight_layout(pad=5.0)

#     plt.axis('off')
#     target = "predictions/" + classes[0]
    target = "predictions"
#     print(target)
    if not os.path.exists(target): os.mkdir(target)
        
    top_class_index = np.argmax(probabilities)
    top_class = classes[top_class_index]
    print(top_class)
        
    save_path = target + "/ternary_prediction_" + top_class + "_" + \
                        "{:d}".format(int(probabilities[top_class_index]*100)) + "pc_" + \
                        image_path
    print(save_path)
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
# bbox_inches='tight'

In [ ]:
import sys
image_folder = "/mnt/hd2/images/all/"

for i, image_path in enumerate(filepaths[0:1]):
#     if i % 500 == 0:
#         end = time.time()
#         elap = end - start;
#         print("analyzing image %d / %d. Time: %4.4f seconds." % (i, len(images),elap))
#         start = time.time()
#     fig, ax = plt.subplots()
#     fig.set_size_inches(20, 10)

    img, x = load_image(image_folder + image_path)
    im = mpimg.imread(image_folder + image_path)
    width, height = pim.size
    print(width, height)
    asp = width/height
    
    pim = Image.open(image_folder + image_path)
    
#     out = Image.new('RGB', (720, 720), color = (200, 200, 200))
#     fnt = ImageFont.truetype("Pillow/Tests/fonts/FreeMono.ttf", 40)
#     d = ImageDraw.Draw(out)
    d = ImageDraw.Draw(pim)
    
#     out.show()
    pim.save(sys.stdout, "PNG")
    
    prediction = model.predict(x)
    
    classes = []
    probabilities = []
    
    for i, (_, pred, prob) in enumerate(decode_predictions(prediction)[0]):
#         print("predicted %s with probability %0.3f" % (pred, prob))
        classes.append(pred)
        probabilities.append(prob)
        
    for i, (c, p) in enumerate(zip(classes, probabilities)):
        y = i * 30 + 50
#         x = width 
        s = str(c) + " : "
#         plt.text(width + 100, y, s, horizontalalignment='right', fontsize=18)
#         plt.text(width + 170, y, "{:.2f} %".format(p*100), 
#                  horizontalalignment='right', fontsize=18)